# Langsmith

In [24]:

%pip install -qU langsmith

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
     

import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
     


In [23]:


%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a knowledgeable assistant for geospatial datasets. Please provide answers based only on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])

# model
model = ChatOpenAI(model="gpt-3.5-turbo")

# Define the output parser
output_parser = StrOutputParser()  # Parsing output into string format

# chain creation
chain = prompt | model | output_parser

# sample question and context
question = "Which cities are associated with the tile SENTINEL2A_20191116_103704_633?"
context = """
Tile SENTINEL2A_20191116_103704_633 is associated with the following cities:
- Frankfurt
- Darmstadt
- Wiesbaden
"""

# Invoking the chain with error handling
try:
    response = chain.invoke({"question": question, "context": context})

    print(response)
except Exception as e:
    print(f"An error occurred: {e}")

     

The cities associated with the tile SENTINEL2A_20191116_103704_633 are Frankfurt, Darmstadt, and Wiesbaden.


# Semantic Model Extraction

In [6]:
%pip install -q langchain-community langchain-openai langchain_experimental neo4j
     

In [21]:
# Installing the neo4j package
%pip install neo4j

from langchain.graphs import Neo4jGraph

url = "bolt://localhost:7687"
username = "neo4j"
password = "12341234"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)
    

In [9]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
     

     


In [11]:
metadata_text = """
Nodes:
Product: name="Sentinel-2 L2A Maja"
Organization: name="German Aerospace Center (DLR)"
Person: name="Pablo Angelo"
ReferenceSystem: name="EPSG:4326 (WGS 84)"
Keyword: text="Satellite"
Keyword: text="Earth Observation"
Keyword: text="Multispectral"
Keyword: text="Sentinel-2"
Keyword: text="MSI"
Keyword: text="Level 2A"
Keyword: text="MAJA"
Keyword: text="Atmospheric Correction"
Keyword: text="Cloud Detection"
DownloadLink: url="https://download.geoservice.dlr.de/S2_L2A_MAJA/", description="Map-based Download (S2 MAJA)"
DownloadLink: url="https://download.geoservice.dlr.de/S2_L2A_MAJA/files/", description="HTTP Download (S2 MAJA)"
SensorType: name="Optical"
Date: "2024-07-12T09:25:33"
ISOStandard: name="ISO 19115 : 2003/Cor.1:2006"
OrbitType: name="LEO"
DOI: "10.15489/ifczsszkcp63"
JSON: url="https://geoservice.dlr.de/eoc/ogc/stac/v1/collections/S2_L2A_MAJA/items?f=application/geo%2Bjson"
GeoJSON: url="https://geoservice.dlr.de/eoc/oseo/search?identifier=S2_L2A_MAJA&httpAccept=application/geo%2Bjson"
Visualization: url="https://geoservice.dlr.de/web/maps/sentinel2:l2a:maja"
Tile: name="SENTINEL2A_20191116-103650-357_L2A_T32UMB_C_V1-2"
Tile: name="SENTINEL2A_20191116-103708-854_L2A_T32ULA_C_V1-2"
Tile: name="SENTINEL2A_20191116-103704-633_L2A_T32UMA_C_V1-2"
Tile: name="SENTINEL2A_20191116-103737-445_L2A_T32ULU_C_V1-2"
Tile: name="SENTINEL2A_20191116-103751-825_L2A_T32TLT_C_V1-2"
Tile: name="SENTINEL2A_20191116-103742-760_L2A_T32TNT_C_V1-2"
Tile: name="SENTINEL2A_20191116-103733-256_L2A_T32UMU_C_V1-2"

Relationships:

(Product)-[:CONTACT_INFORMATION]->(Organization)
(Product)-[:POINT_OF_CONTACT]->(Person)
(Product)-[:REFERENCE_SYSTEM]->(ReferenceSystem)
(Product)-[:DESCRIBED_BY]->(Keyword)
(Product)-[:HAS_DOWNLOAD_LINK]->(DownloadLink)
(Product)-[:TEMPORAL_EXTENT]->(Date)
(Product)-[:SPATIAL_EXTENT]->(SpatialExtent) 
(Product)-[:HAS_SENSOR_TYPE]->(SensorType)
(Product)-[:HAS_ISO_STANDARD]->(ISOStandard)
(Product)-[:HAS_ORBIT_TYPE]->(OrbitType)
(Product)-[:HAS_DOI]->(DOI)
(Product)-[:HAS_JSON]->(JSON)
(Product)-[:HAS_GEOJSON]->(GeoJSON)
(Product)-[:HAS_VISUALIZATION]->(Visualization)
(Product)-[:HAS_TILES]->(Tile)
"""


In [12]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4o-2024-08-06")

llm_transformer = LLMGraphTransformer(llm=llm) # documentation, see https://python.langchain.com/docs/how_to/graph_constructing/
     


In [13]:
from langchain_core.documents import Document

documents = [Document(page_content=metadata_text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")
     

Nodes:[Node(id='Sentinel-2 L2A Maja', type='Product', properties={}), Node(id='German Aerospace Center (Dlr)', type='Organization', properties={}), Node(id='Pablo Angelo', type='Person', properties={}), Node(id='Epsg:4326 (Wgs 84)', type='Referencesystem', properties={}), Node(id='Satellite', type='Keyword', properties={}), Node(id='Earth Observation', type='Keyword', properties={}), Node(id='Multispectral', type='Keyword', properties={}), Node(id='Sentinel-2', type='Keyword', properties={}), Node(id='Msi', type='Keyword', properties={}), Node(id='Level 2A', type='Keyword', properties={}), Node(id='Maja', type='Keyword', properties={}), Node(id='Atmospheric Correction', type='Keyword', properties={}), Node(id='Cloud Detection', type='Keyword', properties={}), Node(id='Https://Download.Geoservice.Dlr.De/S2_L2A_Maja/', type='Downloadlink', properties={}), Node(id='Https://Download.Geoservice.Dlr.De/S2_L2A_Maja/Files/', type='Downloadlink', properties={}), Node(id='Optical', type='Sensort

In [14]:
graph.add_graph_documents(graph_documents)
     

# Graph QA using GraphCypherQAChain

In [22]:
%pip install  --quiet langchain langchain-openai langchain-community neo4j
     

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain.graphs import Neo4jGraph
url = "bolt://localhost:7687"
username = "neo4j"
password = "12341234"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)
    

In [18]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
     

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
import os

chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-4o-mini"), # gpt-4o-mini	gpt-3.5-turbo
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    verbose=True,
    allow_dangerous_requests=True
)

In [90]:
import gradio as gr

# Defining questions
questions = {
    "question_1": "What is the sensor type of Sentinel-2 L2A Maja Products",
    "question_2": "Describe the Sentinel-2 L2A Maja Products",
    "question_3": "What is the temporal extent of Sentinel-2 L2A Maja Products?",
    "question_4": "Give me the download links for Sentinel-2 L2A Maja Products?",
    "question_5": "What is the DOI for Sentinel-2 L2A Maja Products?",
    "question_6": "What is the reference system used for Sentinel-2 L2A Maja Products?",
    "question_7": "What is the ISO standards for Sentinel-2 L2A Maja Products?",
    "question_8": "List the tiles covered by Sentinel-2 L2A Maja Products?",
    "question_9": "What is the orbit type of Sentinel-2 L2A Maja Products?",
    "question_10": "Provide the GeoJSON of Sentinel-2 L2A Maja Products?",
    "question_11": "Provide the JSON of Sentinel-2 L2A Maja Products?",
    "question_12": "Provide the visualization link of Sentinel-2 L2A Maja Products?",
    "question_13": "Provide all the keywords of Sentinel-2 L2A Maja Products?",
}

# Function to get the answer for a given question
def get_answer(messages):
    question = messages[-1][0]  # Get the latest user message
    question_key = next((key for key, value in questions.items() if value.lower() == question.lower()), None)
    try:
        if question_key:
            response = chain.invoke(questions[question_key])
        else:
            response = chain.invoke(question)
        
        if isinstance(response, dict) and 'result' in response:
            return messages[:-1] + [[question, response['result'].strip()]]  # Replace last user entry with answer
        else:
            return messages[:-1] + [[question, "No result found or invalid response format."]]
    except Exception as e:
        return messages[:-1] + [[question, f"An error occurred while processing your question: {str(e)}"]]

# Like/Dislike interaction handler
def print_like_dislike(data: gr.LikeData):
    print(f"Message index: {data.index}, Liked: {data.liked}, Content: {data.value}")

# Gradio chat interface
with gr.Blocks() as iface:
    gr.Markdown("# Sentinel-2 L2A Maja Products Q&A Chat Interface")
    chatbot = gr.Chatbot(label="Chat History", avatar_images=(
            None,
            "https://em-content.zobj.net/source/twitter/376/hugging-face_1f917.png",))
    user_input = gr.Textbox(placeholder="Ask your question here...", label="Your Question")
    submit_button = gr.Button("Send")
    
    # Interaction function
    def chat_interaction(messages, user_query):
        if user_query:
            # Add user query to the chat history and fetch the answer
            messages = messages + [[user_query, None]]
            return get_answer(messages), ""  # Reset input box
        return messages, ""

    # Submit user input and process bot response
    chat_msg = user_input.submit(chat_interaction, [chatbot, user_input], [chatbot, user_input])
    chat_msg.then(lambda: gr.Textbox(interactive=True), None, [user_input])

    # Like/Dislike feature
    chatbot.like(print_like_dislike, None, None, like_user_message=True)

    # Link components
    submit_button.click(
        chat_interaction, 
        inputs=[chatbot, user_input], 
        outputs=[chatbot, user_input]
    )
    user_input.submit(
        chat_interaction, 
        inputs=[chatbot, user_input], 
        outputs=[chatbot, user_input]
    )

if __name__ == '__main__':
    iface.launch()


/Users/iprincetech/Desktop/Raster-QuestionAnsweringAI/.venv/lib/python3.13/site-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7906

To create a public link, set `share=True` in `launch()`.




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Product)-[:HAS_TILES]->(t:Tile)
WHERE p.id CONTAINS 'Sentinel-2 L2A Maja'
RETURN t

Full Context:
[{'t': {'id': 'Sentinel2A_20191116-103650-357_L2A_T32Umb_C_V1-2'}}, {'t': {'id': 'Sentinel2A_20191116-103708-854_L2A_T32Ula_C_V1-2'}}, {'t': {'id': 'Sentinel2A_20191116-103704-633_L2A_T32Uma_C_V1-2'}}, {'t': {'id': 'Sentinel2A_20191116-103737-445_L2A_T32Ulu_C_V1-2'}}, {'t': {'id': 'Sentinel2A_20191116-103751-825_L2A_T32Tlt_C_V1-2'}}, {'t': {'id': 'Sentinel2A_20191116-103742-760_L2A_T32Tnt_C_V1-2'}}, {'t': {'id': 'Sentinel2A_20191116-103733-256_L2A_T32Umu_C_V1-2'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:Product {id: 'Sentinel-2 L2A Maja'})-[:HAS_DOI]->(d:Doi)
RETURN d

Full Context:
[{'d': {'id': '10.15489/Ifczsszkcp63'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product {id: 'Sentinel-2 L2A Maja'})-[:HAS_DOI]-